In [1]:
import numpy as np
import pickle
from sklearn import linear_model
from sklearn.metrics import roc_auc_score, classification_report, roc_curve

In [2]:
input_path = "E:/CS_Master_Degree_UIUC/CS598_DeepLearning_for_Health_Data/Project/paper290/Processed_data/"
output_path = "E:/CS_Master_Degree_UIUC/CS598_DeepLearning_for_Health_Data/Project/paper290/Processed_data/"

In [3]:
vocabsize_icd = 4780
vocabsize_meds = 3202
vocabsize_labs = 545
vocabsize = vocabsize_icd+vocabsize_meds+vocabsize_labs

In [4]:
picklefile = "ICD_seq.pkl"
with open(input_path + picklefile, "rb") as pkl_rb_obj:
    ICD_seqs = pickle.load(pkl_rb_obj)
    
picklefile = "ICD_codes_all.pkl"
with open(input_path + picklefile, "rb") as pkl_rb_obj:
    ICDs_all = pickle.load(pkl_rb_obj)
    
ICD_seqs[0] #First patent and all ICD codes of all visits of this patient

[['4239', '5119', '78551', '4589', '311', '7220', '71946', '2724'],
 ['7455', '45829', 'V1259', '2724']]

In [5]:
picklefile = "Med_seq.pkl"
with open(input_path + picklefile, "rb") as pkl_rb_obj:
    Med_seqs = pickle.load(pkl_rb_obj)
    
picklefile = "Meds_all.pkl"
with open(input_path + picklefile, "rb") as pkl_rb_obj:
    Meds_all = pickle.load(pkl_rb_obj)

Med_seqs[0][0][0:10] #First patient, first visit and the first 10 prescribed medication

['Glycopyrrolate',
 'Neostigmine',
 'Calcium Gluconate',
 'Magnesium Sulfate',
 'Meperidine',
 'Oxycodone-Acetaminophen',
 'Sucralfate',
 'Acetaminophen',
 'Acetaminophen',
 'Aspirin EC']

In [6]:
picklefile = "Lab_seq.pkl"
with open(input_path + picklefile, "rb") as pkl_rb_obj:
    Lab_seqs = pickle.load(pkl_rb_obj)

picklefile = "labs_all.pkl"
with open(input_path + picklefile, "rb") as pkl_rb_obj:
    labs_all = pickle.load(pkl_rb_obj)
    
Lab_seqs[0][0][0:10] #First patient, first visit and the first 10 lab results

['pH',
 'pH',
 'pH',
 'pH',
 'pH',
 'pH',
 'SPECIMEN TYPE',
 'SPECIMEN TYPE',
 'SPECIMEN TYPE',
 'SPECIMEN TYPE']

In [7]:
picklefile = "mort_lables.pkl"
with open(input_path + picklefile, "rb") as pkl_rb_obj:
    mort_seqs = pickle.load(pkl_rb_obj)
    
mort_seqs[0:10]

[0, 1, 0, 0, 0, 1, 1, 0, 1, 0]

In [8]:
# fout = open("logistic_regression_interpretations.txt", 'w')
# For each patient 
def combine_encounter(seqs, vocab):
    vocab_size = len(vocab)
    enc_size = len(seqs)
    ret_vector = np.zeros((enc_size, vocab_size))
    for i, enc in enumerate(seqs):
        for code in enc:
            if code != -1:
                j = vocab.index(code)
                ret_vector[i,j] = 1
    ret_vector = ret_vector.mean(axis = 0)
    return ret_vector

In [9]:
num_patient =  len(ICD_seqs)

icd_features = np.zeros((num_patient, len(ICDs_all)))
med_features = np.zeros((num_patient, len(Meds_all)))
lab_features = np.zeros((num_patient, len(labs_all)))
                        
for i in range(num_patient):
    icd_features[i] = combine_encounter(ICD_seqs[i], ICDs_all)
    med_features[i] = combine_encounter(Med_seqs[i], Meds_all)
    lab_features[i] = combine_encounter(Lab_seqs[i], labs_all)
    
input_seqs = np.concatenate((icd_features,med_features,lab_features),axis = 1)    
# input_seqs = np.array([np.concatenate((combine_encounter(ICD_seqs[i], ICDs_all), combine_encounter(Med_seqs[i], Meds_all), combine_encounter(Lab_seqs[i], labs_all)), axis=0) for i in range(num_patient)])

In [10]:
np.save(output_path+'icd_inputs.npy',icd_features)
np.save(output_path+'med_inputs.npy',med_features)
np.save(output_path+'lab_inputs.npy',lab_features)
np.save(output_path+'input_seqs.npy',input_seqs)